In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from census import Census
import requests
import json
from uszipcode import SearchEngine
import gmaps

# Import census key and set to year 2016
from config_census import census_key
c = Census(census_key, year=2016)

# Import weather API key
from config_maps import api_key

#Import gmaps API key
from config_g import gkey

In [2]:
# Run Census Search to retrieve data on by states
census_data = c.acs5.get(("NAME", "B01003_001E", "B17001_002E", "B15003_002E","B15003_002E", 
                          "B15003_017E", "B15003_018E", "B15003_021E", "B15003_022E", "B15003_023E", 
                          "B15003_024E", "B15003_025E"), {'for': 'zip code tabulation area:*'})

# Create datafram
census_df = pd.DataFrame(census_data)

# Relabel and Reorder Columns
census_df = census_df.rename(columns={"zip code tabulation area": "Zipcode",
                                      "NAME": "Name",
                                      "B01003_001E": "Population",
                                      "B17001_002E": "Poverty Count",
                                      "B15003_002E": "No Education", 
                                      "B15003_017E": "High School Education", 
                                      "B15003_018E": "GED Education",
                                      "B15003_021E": "Associates Degree",
                                      "B15003_022E": "Bachelors Degree", 
                                      "B15003_023E": "Masters Degree",
                                      "B15003_024E": "Professional Degree",
                                      "B15003_025E": "Doctoral Degree"
                                      })

census_df = census_df.reindex(columns=["Zipcode", "Population", "Poverty Count",  "No Education", "High School Education", 
                                       "GED Education", "Associates Degree", "Bachelors Degree" , "Masters Degree", 
                                       "Professional Degree", "Doctoral Degree"])

# Print
census_df.head()

,Zipcode,Population,Poverty Count,No Education,High School Education,GED Education,Associates Degree,Bachelors Degree,Masters Degree,Professional Degree,Doctoral Degree
0,83202,23192.0,3504.0,108.0,3432.0,688.0,1539.0,2131.0,636.0,97.0,166.0
1,83203,279.0,83.0,0.0,34.0,28.0,13.0,16.0,0.0,0.0,0.0
2,83204,18071.0,3781.0,69.0,2654.0,698.0,866.0,2212.0,661.0,324.0,295.0
3,83209,1165.0,174.0,12.0,0.0,0.0,14.0,32.0,25.0,0.0,13.0
4,83210,3175.0,409.0,45.0,541.0,86.0,143.0,117.0,73.0,0.0,5.0


In [3]:
# Create No Degree column
census_df["No Degree"] = census_df["No Education"] + census_df["High School Education"] + census_df["GED Education"]

# Create Degree column
census_df["Degree"] = census_df["Associates Degree"] + census_df["Bachelors Degree"] + census_df["Masters Degree"] \
                    + census_df["Professional Degree"] + census_df["Doctoral Degree"]

# All columns needed for data by state saved to .csv in case needed
census_df.to_csv("Census_by_zip_all.csv", index=False)

# Print
census_df.head()

,Zipcode,Population,Poverty Count,No Education,High School Education,GED Education,Associates Degree,Bachelors Degree,Masters Degree,Professional Degree,Doctoral Degree,No Degree,Degree
0,83202,23192.0,3504.0,108.0,3432.0,688.0,1539.0,2131.0,636.0,97.0,166.0,4228.0,4569.0
1,83203,279.0,83.0,0.0,34.0,28.0,13.0,16.0,0.0,0.0,0.0,62.0,29.0
2,83204,18071.0,3781.0,69.0,2654.0,698.0,866.0,2212.0,661.0,324.0,295.0,3421.0,4358.0
3,83209,1165.0,174.0,12.0,0.0,0.0,14.0,32.0,25.0,0.0,13.0,12.0,84.0
4,83210,3175.0,409.0,45.0,541.0,86.0,143.0,117.0,73.0,0.0,5.0,672.0,338.0


In [4]:
# Only use necessary columns
census_df = census_df.reindex(columns=["Zipcode", "Population", "Poverty Count", "No Degree", "Degree"])
census_df.head()

,Zipcode,Population,Poverty Count,No Degree,Degree
0,83202,23192.0,3504.0,4228.0,4569.0
1,83203,279.0,83.0,62.0,29.0
2,83204,18071.0,3781.0,3421.0,4358.0
3,83209,1165.0,174.0,12.0,84.0
4,83210,3175.0,409.0,672.0,338.0


In [5]:
# Create new columns for Lat on Lng
census_df["County"] = ""

census_df.head()

,Zipcode,Population,Poverty Count,No Degree,Degree,County
0,83202,23192.0,3504.0,4228.0,4569.0,
1,83203,279.0,83.0,62.0,29.0,
2,83204,18071.0,3781.0,3421.0,4358.0,
3,83209,1165.0,174.0,12.0,84.0,
4,83210,3175.0,409.0,672.0,338.0,


In [6]:
# Add search from uszipcode dependency
search = SearchEngine(simple_zipcode=True)

# Add city of each zipcode
for index, row in census_df.iterrows():
    target_zip = row["Zipcode"]
    zipcode = search.by_zipcode(target_zip)
    
    try:
        census_df.loc[index, "County"] = zipcode.to_dict()['county']
                
    except (KeyError):
        print("Zip not found. Skipping...")
        
census_df.head()

,Zipcode,Population,Poverty Count,No Degree,Degree,County
0,83202,23192.0,3504.0,4228.0,4569.0,Bannock County
1,83203,279.0,83.0,62.0,29.0,Bannock County
2,83204,18071.0,3781.0,3421.0,4358.0,Bannock County
3,83209,1165.0,174.0,12.0,84.0,Bannock County
4,83210,3175.0,409.0,672.0,338.0,Bingham County


In [7]:
county_df = census_df.groupby("County").sum()
county_df = county_df.reset_index()
county_df["Lat"] = ""
county_df["Lng"] = ""
county_df.head()

,County,Population,Poverty Count,No Degree,Degree,Lat,Lng
0,,5069.0,592.0,1421.0,648.0,,
1,Aasco Municipio,27976.0,13734.0,6757.0,4612.0,,
2,Abbeville County,20747.0,4669.0,5102.0,3220.0,,
3,Acadia Parish,61112.0,12667.0,16289.0,6807.0,,
4,Accomack County,33060.0,6341.0,9936.0,5802.0,,


In [8]:
# Use openweathermap API to import Lat and Lng of Cities
base_url = "http://api.openweathermap.org/data/2.5/weather?q="

for index, row in county_df.iterrows():
    target_county = row["County"]
    
    query = f"{base_url}{target_county}&appid={api_key}"
    response = requests.get(query).json()
    
    try:
        county_df.loc[index, "Lat"] = response["coord"]["lat"]
        county_df.loc[index, "Lng"] = response["coord"]["lon"]

    except (KeyError):
        pass

In [9]:
# Delete blank rows
county_df = county_df[county_df.County != '']
county_df = county_df[county_df.Lat != '']
county_df.head()

,County,Population,Poverty Count,No Degree,Degree,Lat,Lng
2,Abbeville County,20747.0,4669.0,5102.0,3220.0,34.23,-82.47
5,Ada County,429367.0,51360.0,64301.0,131300.0,43.42,-116.13
6,Adair County,72700.0,17031.0,17784.0,11932.0,40.2,-92.6
7,Adams County,768317.0,100228.0,172132.0,149855.0,39.87,-104.38
8,Addison County,35684.0,2861.0,7780.0,10793.0,44.05,-73.12


In [10]:
# New columns needed for data by county saved to .csv in case needed
# CSV is used in Map_by_County
county_df.to_csv("Census_by_county.csv", index=False)